<span style="color:red">Work in progress</span>

--> This jupyter notebook serve as the basic structure to perform life cycle assessment within our research group (Used at least by Augustin & Robin).

# Introduction

The architecture of this notebook is explained in the Figure bellow that depicts the three different type of brightway2 (BW2) databases that are used in the notebook:
- the biosphere database from ecoinvent
- the original ecoinvent database version 3.9.1 cutoff
- an Open Source (OS) database where custom activities are created.

This jupyter notebook does the following:
1. Create the BW2 environment to compute the LCA
2. Generate the foreground activities, custom activities and modified activies in the framework of LCA algebraic using several excel file as data input
3. Compute LCIA results with uncertainty and distribution based on parameter distributions.

The following figure illustrates the database structure of this BW2 notebook and the purpose of the various files used.

![title](image/figure_notebook_structure.png)


# Importing relevant packages

In [ ]:
import brightway2 as bw
import lca_algebraic as agb
from sympy import init_printing
import bw2io
from dotenv import load_dotenv
import pandas as pd
from sympy import symbols
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os 
import json
from importlib import reload

In [ ]:
# Custom packages
from src.ei_access import EI_Access
from src.ei_access.setup import setup_ecoinvent_database

from src.utils.sheets_parser import find_activity,process_parameters,create_custom_activities,create_foreground

# Parameters

In [ ]:

#### Project & ecoinvent database parameters
project_name= 'ECS-LCA'

ei_acc = EI_Access()

#### Life cycle data parameters
xlsx_path = "./sheets/"
# This Excel file must be formatted according to the provided template.
LC_data_Foreground_file_path = xlsx_path + "LC_data_Foreground.xlsx"
foreground_list=["foreground_ICT_2024_BE"] #These foreground can represent different scenario for example

#Specify the files that contain the data to build the custom and modified activities.
custom_meta_data= xlsx_path + "custom_meta_data.xlsx"

# Specify the Foreground for which the results will be computed
selected_foreground = 'foreground_ICT_2024_BE'

# Specify the LCIA method with which the results will be computed
impacts_GWP_specific=('EF v3.1', 'climate change', 'global warming potential (GWP100)')
#impacts_GWP_specific=('CML v4.8 2016 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
impacts_ADP_specific=('EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)')
LCIA_method_list=[impacts_GWP_specific,impacts_ADP_specific]
#Specify amoung which label the contribution analysis should be performed:
contribution_analysis_label = "LC phase"


#Specify the output folder to export the results
output_folder = r"./results"

In [ ]:
bw.projects

# Initialize the project and the ecoinvent database

In [ ]:
# Uncomment and run only if you think you messed up your project
#bw.projects.delete_project(name=project_name, delete_dir=True)

In [ ]:
bw.projects.set_current(project_name) # Set the current project, can be any name

In [ ]:
if 'biosphere3' not in bw.databases.keys():
    #Content of bw2.setup() function but with overwrite=True
    bw.create_default_biosphere3()
    bw.create_default_lcia_methods(overwrite=True)
    bw.create_core_migrations()

In [ ]:
setup_ecoinvent_database(ei_acc)

In [ ]:
agb.list_databases()

# Load the life-cycle data (foreground systems, created and modified activities)

In [ ]:
# Loading the foreground systems in a dictionnary
foregrounds_All = {}
for foreground_name in foreground_list:
    foreground=pd.read_excel(LC_data_Foreground_file_path, sheet_name=foreground_name)
    foregrounds_All[foreground_name]=foreground

In [ ]:
custom_meta_data_DF=pd.read_excel(custom_meta_data)
custom_meta_data_DF=custom_meta_data_DF.set_index('sheet name')

In [ ]:
# Creating a dataframe for each custom activity (custom activity = green sheets)
OS_database_dataframes = {}

for index,row in custom_meta_data_DF.iterrows():
    sheet_data = pd.read_excel(xlsx_path + row["file"], sheet_name=index)  
    OS_database_dataframes[index] = sheet_data

In [ ]:
agb.resetParams()      # Reset parameters 

In [ ]:
parameter_registry = {} # Add parameters from custom and modified activity to the parameter_registry

for sheet_name, sheet_df in OS_database_dataframes.items():
    print(f"Processing parameters from sheet: {sheet_name}")
    process_parameters(sheet_df, parameter_registry,sheet_name)  # create new parameters for custom activities

# Defining the foreground

In [ ]:
OS_database="OS database"

agb.resetDb(OS_database)
agb.setForeground(OS_database) #Create one database where all custom and modified activities will be added.
agb.list_databases()        # Sanity check - All activities reset to zero ?

In [ ]:
process_parameters(foregrounds_All[f"{selected_foreground}"], parameter_registry,selected_foreground) # create new parameters for foreground activities and add them to the parameter_registry

# Create custom activities

In [ ]:
create_custom_activities(custom_meta_data_DF,OS_database_dataframes,OS_database,parameter_registry)

In [ ]:
agb.list_databases() #Should be non zero for the foreground custom DB

In [ ]:
create_foreground(foregrounds_All,selected_foreground,OS_database,parameter_registry)

In [ ]:
agb.list_databases() #Should be non zero for the foreground custom DB

# Foreground generation

In [ ]:
Exchanges_Foreground = {} # Initialize an empty dictionary to store exchanges

for index, row in foregrounds_All[f"{selected_foreground}"].iterrows(): # Iterate through the main sheet to collect activities for the exchanges
    Exchanges_Foreground[agb.findActivity(row["LCA algebraic name"], db_name=OS_database, loc="GLO")]=1

print(Exchanges_Foreground)
agb.newActivity(OS_database,selected_foreground,  "unit", exchanges=Exchanges_Foreground) # Create the foreground

In [ ]:
total_foreground_exchanges = {}
total_foreground_exchanges[agb.findActivity(selected_foreground, db_name=OS_database, loc="GLO")] = 1

# Reference Flow

In [ ]:
reference_flow = agb.newActivity(OS_database, "reference flow", "unit", exchanges=total_foreground_exchanges)

# Impact Methods Choice

In [ ]:
# Retrieve impact categories (here from EF v3.1)
impacts_GWP = agb.findMethods(search="climate change", mainCat="EF v3.1") # GWP impact categories from EF 3.1
impacts_GWP_other = agb.findMethods(search="climate change") # GWP impact categories from EF 3.1
impacts_ADP = agb.findMethods(search="ADP", mainCat="EF v3.1") # GWP impact categories from EF 3.1
impacts_all = agb.findMethods(search="", mainCat="EF v3.1")               # All impact categories from EF 3.1

# Compute Absolute Impacts

In [ ]:
absolute_impacts = agb.compute_impacts(reference_flow, LCIA_method_list, functional_unit= 1)
absolute_impacts

# Contribution Analysis

In [ ]:
for index, row in foregrounds_All[f"{selected_foreground}"].iterrows():
    activity_name = row["LCA algebraic name"]
    sub_assembly_label = row[contribution_analysis_label]  # choosing labels for activities -> life cycle phases : results per phase, 
    agb.findActivity(activity_name, db_name=OS_database, loc="GLO").updateMeta(phase=sub_assembly_label, label=sub_assembly_label)
print("Finished labeling activities.")

df_impacts_axis = agb.compute_impacts(reference_flow, LCIA_method_list, functional_unit=1, axis="label")
df_impacts_axis_ = df_impacts_axis.drop(index=['*sum*'])     # Drop the 'sum' 
df_sums = df_impacts_axis_.sum(axis=0)                       # Compute sums 
df_normalized = df_impacts_axis_.div(df_sums, axis=1) * 100  # Normalize data

In [ ]:
### ------------------------------------------------------------------
# 1.  Set-up
# ------------------------------------------------------------------
df = df_impacts_axis.drop(index="*sum*")                    # convenience alias
labels = df.index                       # contribution labels (rows)
categories = df.columns                 # the two impact categories
palette = sns.color_palette("Spectral", n_colors=len(labels))

# ------------------------------------------------------------------
# 2.  Create a 1-by-N grid of sub-plots (one per impact category)
# ------------------------------------------------------------------
fig, axes = plt.subplots(
    nrows=1,
    ncols=len(categories),
    figsize=(10, 3),
    sharey=True,                 # keep the single y tick aligned
    constrained_layout=True
)

# ------------------------------------------------------------------
# 3.  Draw a stacked horizontal bar in *each* subplot
# ------------------------------------------------------------------
for ax, cat in zip(axes, categories):
    left = 0
    for color, (label, value) in zip(palette, df[cat].items()):
        ax.barh(
            y=0,                 # only one device
            width=value,
            left=left,
            height=0.6,
            color=color,
            label=label
        )
        left += value            # accumulate for stacking

    # --- Cosmetics -------------------------------------------------
    # Shorten the subplot title to everything before the first " - "
    ax.set_title(cat.split(' - ')[0], fontsize=11)

    # Label the x-axis with the unit text (text inside the brackets)
    unit = cat.split('[')[-1].rstrip(']')
    ax.set_xlabel(unit)
    ax.set_yticks([0])

# ------------------------------------------------------------------
# 4.  One shared legend and overall title
# ------------------------------------------------------------------
fig.legend(labels, loc='upper right', ncol=1, bbox_to_anchor=(0.97, 0.9))
#fig.suptitle('Contribution analysis — single device', y=1.15, fontsize=14)
#plt.savefig(os.path.join(output_folder,"fig_"+contribution_analysis_label+"_"+selected_foreground+".png"))
plt.show()

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(12, 10))
dark_palette = sns.color_palette("Spectral", n_colors=len(df_normalized.index))
df_normalized.T.plot(kind='bar', stacked=True, ax=ax, width=0.8, color=dark_palette)
#dark_palette = sns.color_palette("Spectral", n_colors=len(df_impacts_axis_.index))
#df_impacts_axis_.T.plot(kind='bar', stacked=True, ax=ax, width=0.8, color=dark_palette)

# Customize plot
ax.set_title('Impact Categories with Contributions from Selected Labels (Normalized to 100%)')
ax.set_ylabel('Percentage (%)')
ax.set_xticklabels(df_impacts_axis_.columns, rotation=45, ha='right')
ax.legend(title="Labels", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
#plt.savefig(os.path.join(output_folder,"figure","fig_normalized_"+contribution_analysis_label+"_"+selected_foreground+".png"))
plt.show()

# Monte-Carlo Analysis

In [ ]:
MC_runs=agb.incer_stochastic_violin(reference_flow, LCIA_method_list, functional_unit=1, figspace=(0.5,0.5), n=512, figsize=(8, 8), sharex=True,  nb_cols=2)

# Exporting results

In [ ]:
#output_file = os.path.join(output_folder,  selected_foreground+"_contribution_results.xlsx")
#df_impacts_axis.to_excel(output_file, index=True)
#print(f"Impact saved to {output_file}")